In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Input
from tensorflow.keras.layers import Conv1D, Attention, Reshape
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf

In [69]:
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("deep")
plt.rcParams['figure.figsize'] = [14, 8]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True

In [70]:
np.random.seed(42)
tf.random.set_seed(42)

In [71]:
df = pd.read_csv('data/PJME_hourly.csv')

In [72]:
df['Datetime'] = pd.to_datetime(df['Datetime'])

df = df.set_index('Datetime').sort_index() # sắp xếp theo thời gian để chuẩn hóa dữ liệu
df = df.resample('h').mean() # ấy trung bình nếu có nhiều điểm trong cùng 1 giờ

df['PJME_MW'] = df['PJME_MW'].interpolate(method='time') # nội suy giá trị thiếu theo thời gian

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145392 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Freq: h
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PJME_MW  145392 non-null  float64
dtypes: float64(1)
memory usage: 2.2 MB


In [75]:
# Chia tập dữ liệu
train_size = int(len(df) * 0.7)
val_size = int(len(df) * 0.15)

df_train = df.iloc[:train_size].copy()
df_val = df.iloc[train_size:train_size+val_size].copy()
df_test = df.iloc[train_size+val_size:].copy()

print(f"Kích thước tập train: {len(df_train)}")
print(f"Kích thước tập val: {len(df_val)}")
print(f"Kích thước tập test: {len(df_test)}")

Kích thước tập train: 101774
Kích thước tập val: 21808
Kích thước tập test: 21810


In [76]:
# Hàm tạo đặc trưng mới
def feature_engineering(df):
    df = df.copy()
    
    # Đặc trưng độ trễ
    for lag in [1, 24, 168]:
        df[f'lag_{lag}h'] = df['PJME_MW'].shift(lag)
    
    # Đặc trưng thống kê trượt
    for window in [24]:
        df[f'rolling_mean_{window}h'] = df['PJME_MW'].rolling(window=window).mean()
        df[f'rolling_std_{window}h'] = df['PJME_MW'].rolling(window=window).std()
        df[f'rolling_min_{window}h'] = df['PJME_MW'].rolling(window=window).min()
        df[f'rolling_max_{window}h'] = df['PJME_MW'].rolling(window=window).max()
    
    # Differencing
    df['diff_1h'] = df['PJME_MW'].diff(1)
    df['diff_24h'] = df['PJME_MW'].diff(24)
    
    return df

def fillna(df):
    df = df.copy()
    
    for col in df.columns:
        if col.startswith('lag_'):
            df[col] = df.groupby(df.index.hour)[col].transform(lambda x: x.fillna(x.mean()))
    
    for window in [24]:
        mask = df[f'rolling_mean_{window}h'].isna()
        df.loc[mask, f'rolling_mean_{window}h'] = df.loc[mask, 'PJME_MW']
        df[f'rolling_std_{window}h'] = df[f'rolling_std_{window}h'].fillna(0)
        df.loc[mask, f'rolling_min_{window}h'] = df.loc[mask, 'PJME_MW']
        df.loc[mask, f'rolling_max_{window}h'] = df.loc[mask, 'PJME_MW']
    
    df['diff_1h'] = df['diff_1h'].fillna(0)
    df['diff_24h'] = df['diff_24h'].fillna(0)
    
    return df

In [77]:
df_train_fe = feature_engineering(df_train)
df_val_fe = feature_engineering(df_val)
df_test_fe = feature_engineering(df_test)

df_train_filled = fillna(df_train_fe)
df_val_filled = fillna(df_val_fe)
df_test_filled = fillna(df_test_fe)

In [78]:
def create_time_features(df):
    df = df.copy()
    
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month
    
    # Đặc trưng thời điểm
    df['season'] = ((df['month'] % 12) // 3) / 4  # 0, 0.25, 0.5, 0.75
    
    return df

In [79]:
df_train_processed = create_time_features(df_train_filled)
df_val_processed = create_time_features(df_val_filled)
df_test_processed = create_time_features(df_test_filled)

In [80]:
target_scaler = MinMaxScaler()
df_train_processed['PJME_MW_scaled'] = target_scaler.fit_transform(df_train_processed[['PJME_MW']])
df_val_processed['PJME_MW_scaled'] = target_scaler.transform(df_val_processed[['PJME_MW']])
df_test_processed['PJME_MW_scaled'] = target_scaler.transform(df_test_processed[['PJME_MW']])

In [84]:
seq_length = 24 

# Hàm tạo chuỗi
def create_sequences(data, feature_cols, target_col, seq_length):
    feature_data = data[feature_cols].values
    target_data = data[target_col].values
    
    X, y = [], []
    
    for i in range(len(data) - seq_length):
        X.append(feature_data[i:i+seq_length])
        y.append(target_data[i+seq_length])
    
    return np.array(X), np.array(y)

numeric_features = df_train_processed.columns.tolist()
target_col = 'PJME_MW_scaled'
target_idx = numeric_features.index(target_col)

print(f"Vị trí của target trong danh sách đặc trưng: {target_idx}")

X_train, y_train = create_sequences(df_train_processed, numeric_features, target_col, seq_length)
X_val, y_val = create_sequences(df_val_processed, numeric_features, target_col, seq_length)
X_test, y_test = create_sequences(df_test_processed, numeric_features, target_col, seq_length)

print(f"Hình dạng X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Hình dạng X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"Hình dạng X_test: {X_test.shape}, y_test: {y_test.shape}")

Vị trí của target trong danh sách đặc trưng: 14
Hình dạng X_train: (101750, 24, 15), y_train: (101750,)
Hình dạng X_val: (21784, 24, 15), y_val: (21784,)
Hình dạng X_test: (21786, 24, 15), y_test: (21786,)


In [85]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.0001,
    verbose=1
)

In [87]:
def build_conv_lstm_model(input_shape):
    inputs = Input(shape=input_shape)
    
    conv1 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(inputs)
    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs)

    conv_concat = tf.keras.layers.Concatenate()([conv1, conv2])
    conv_bn = BatchNormalization()(conv_concat)

    lstm1 = LSTM(64, activation='tanh', return_sequences=True, recurrent_dropout=0.01)(conv_bn)
    lstm1 = LayerNormalization()(lstm1)
  
    attention = MultiHeadAttention(num_heads=8, key_dim=32)(lstm1, lstm1)
    attention = Dropout(0.01)(attention)
    attention = LayerNormalization()(attention + lstm1)  
 
    lstm2 = LSTM(64, activation='tanh')(attention)
    lstm2 = Dropout(0.01)(lstm2)

    output = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(lstm2)
    output = Dense(32, activation='relu')(output)
    output = Dropout(0.01)(output)
    output = Dense(1)(output)
    
    model = Model(inputs=inputs, outputs=output)
    
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=0.001,
        decay_steps=10000,
        alpha=0.0001
    )
    optimizer = Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

In [88]:
input_shape = (X_train.shape[1], X_train.shape[2])
print("\nXây dựng mô hình ConvLSTM...")
conv_lstm_model = build_conv_lstm_model(input_shape)
conv_lstm_model.summary()


Xây dựng mô hình ConvLSTM...


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 24, 15)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 24, 128)   │      5,888 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_12 (Conv1D)  │ (None, 24, 64)    │      4,864 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 24, 192)   │          0 │ conv1d_11[0][0],  │
│ (Concatenate)       │                   │            │ conv1d_12[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 24, 192)   │        768 │ concatenate_4[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_10 (LSTM)      │ (None, 24, 64)    │     65,792 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 24, 64)    │        128 │ lstm_10[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 64)    │     66,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_21          │ (None, 24, 64)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 24, 64)    │          0 │ dropout_21[0][0], │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 24, 64)    │        128 │ add_5[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_11 (LSTM)      │ (None, 64)        │     33,024 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_22          │ (None, 64)        │          0 │ lstm_11[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 64)        │      4,160 │ dropout_22[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 32)        │      2,080 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_23          │ (None, 32)        │          0 │ dense_15[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 1)         │         33 │ dropout_23[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 183,233 (715.75 KB)

 Trainable params: 182,849 (714.25 KB)

 Non-trainable params: 384 (1.50 KB)

In [89]:
history = conv_lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

Epoch 1/30
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 128s 39ms/step - loss: 0.0211 - mae: 0.0318 - val_loss: 2.9026e-04 - val_mae: 0.0104 - learning_rate: 7.7058e-04
Epoch 2/30
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 107s 34ms/step - loss: 7.5763e-04 - mae: 0.0198 - val_loss: 1.5778e-04 - val_mae: 0.0079 - learning_rate: 2.9288e-04
Epoch 3/30
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 118s 37ms/step - loss: 5.1656e-04 - mae: 0.0166 - val_loss: 1.1412e-04 - val_mae: 0.0066 - learning_rate: 5.3114e-06
Epoch 4/30
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 128s 40ms/step - loss: 4.2795e-04 - mae: 0.0151 - val_loss: 1.1005e-04 - val_mae: 0.0063 - learning_rate: 1.0000e-07
Epoch 5/30
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 126s 40ms/step - loss: 4.2824e-04 - mae: 0.0151 - val_loss: 1.0957e-04 - val_mae: 0.0063 - learning_rate: 1.0000e-07
Epoch 6/30
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 115s 36ms/step - loss: 4.2785e-04 - mae: 0.0151 - val_loss: 1.0930e-04 - val_mae: 0.0063 - learning_rate: 1.0000e-07
Epoch 7/30
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 121s 38m

In [90]:
# Đánh giá mô hình
def evaluate_model(model, X, y):
    y_pred = model.predict(X)
    
    y_pred_original = target_scaler.inverse_transform(y_pred)
    y_original = target_scaler.inverse_transform(y.reshape(-1, 1))
    
    rmse = np.sqrt(mean_squared_error(y_original, y_pred_original))
    mae = mean_absolute_error(y_original, y_pred_original)
    mape = np.mean(np.abs((y_original - y_pred_original) / y_original)) * 100
    
    print(f"- RMSE: {rmse:.2f}")
    print(f"- MAE: {mae:.2f}")
    print(f"- MAPE: {mape:.2f}%")
    
    return rmse, mae, mape

In [91]:
print("\nĐánh giá mô hình ConvLSTM...")
train_metrics = evaluate_model(conv_lstm_model, X_train, y_train, "huấn luyện")
val_metrics = evaluate_model(conv_lstm_model, X_val, y_val, "xác thực")
test_metrics = evaluate_model(conv_lstm_model, X_test, y_test, "kiểm tra")


Đánh giá mô hình ConvLSTM...
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 26s 8ms/step
Đánh giá trên tập huấn luyện:
- RMSE: 386.64
- MAE: 282.14
- MAPE: 0.87%
681/681 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step
Đánh giá trên tập xác thực:
- RMSE: 404.03
- MAE: 295.21
- MAPE: 0.94%
681/681 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step
Đánh giá trên tập kiểm tra:
- RMSE: 405.34
- MAE: 298.02
- MAPE: 0.96%
